# Visualising Building Damage
# ==========================
The provided Python script is designed for data analysis and visualization within a Jupyter Notebook environment. It begins by setting up the necessary environment, importing libraries, and adjusting the working directory if needed. The script loads a dataset from a CSV file, containing information about buildings impacted by war fires, into a Pandas DataFrame. It then conducts an analysis, displaying unique values of the 'building_count' column. Subsequently, the script creates two Folium maps to visualize the spatial distribution of affected buildings. The first map marks locations where buildings have been impacted by war fires, displaying building counts as pop-up information. The second map highlights areas with war fires and a non-null 'geometry_buff,' using circular markers to indicate the number of affected buildings. Finally, the script converts the Jupyter Notebook to a Python script using the `nbconvert` command. Note that the effectiveness of the script depends on the specifics of the dataset and the intended analysis.

- Number of affected buildings in [Izyum, Lyman, Kherson, Nikopol]
- Average number of affected buildings in each of these Oblasts
- Number of ADM3 areas that border Zone A
- Number of shelling events across these Oblasts

In [1]:
# Auto update notebook imports
#%load_ext autoreload
# %autoreload 2

# Backtrack to folder source directory if it doesn't already exist in path
import os
import sys

if os.path.basename(os.getcwd()) == "notebooks":
    os.chdir("..")

sys.path.append(os.getcwd())
print(os.getcwd())

import geopandas as gpd
import pandas as pd
from shapely.wkt import loads
import numpy as np
from shapely import wkt
from shapely.geometry import Point
import folium
from geopandas import GeoDataFrame

/Users/harryhopalot/Documents/GitHub/dssd_Insulate_ukraine


In [2]:
impacted_buildings_filename = "./data/processed/buildings_impacted_by_war_fires.csv"
geo_warfires = pd.read_csv(impacted_buildings_filename)
geo_warfires.head()

,LATITUDE,LONGITUDE,ADM3_PCODE,municipality,ADM3_EN,ADM3_UA,ADM3_RU,ADM2_EN,ADM2_UA,ADM2_RU,...,ADM1_PCODE,ADM0_EN,year,ACQ_TIME,date,pop_exact,state,geometry,geometry_buff,building_count
0,45.30062,28.89845,UA5108003,UA5108003,Izmailska,Ізмаїльська,Измаильская,Izmailskyi,Ізмаїльський,Измаильский,...,UA51,Ukraine,2023,48,2023-08-02,280.977295,Odesa Oblast,POINT (2300456.403102858 3338797.1530194916),POLYGON ((2300506.403102858 3338797.1530194916...,0.0
1,45.30187,28.90038,UA5108009,UA5108009,Safianivska,Саф'янівська,Сафьяновская,Izmailskyi,Ізмаїльський,Измаильский,...,UA51,Ukraine,2023,1155,2023-08-03,316.327911,Odesa Oblast,POINT (2300546.4663278293 3339036.818291042),POLYGON ((2300596.4663278293 3339036.818291042...,0.0
2,45.30349,28.89518,UA5108003,UA5108003,Izmailska,Ізмаїльська,Измаильская,Izmailskyi,Ізмаїльський,Измаильский,...,UA51,Ukraine,2023,30,2023-08-03,280.977295,Odesa Oblast,POINT (2300803.3471947545 3338470.9640192464),POLYGON ((2300853.3471947545 3338470.964019246...,0.0
3,45.30404,28.89320,UA5108003,UA5108003,Izmailska,Ізмаїльська,Измаильская,Izmailskyi,Ізмаїльський,Измаильский,...,UA51,Ukraine,2023,120,2023-08-03,280.977295,Odesa Oblast,POINT (2300894.4656776898 3338254.431632535),POLYGON ((2300944.4656776898 3338254.431632535...,0.0
4,45.30515,28.90079,UA5108009,UA5108009,Safianivska,Саф'янівська,Сафьяновская,Izmailskyi,Ізмаїльський,Измаильский,...,UA51,Ukraine,2023,48,2023-08-02,316.327911,Odesa Oblast,POINT (2300867.4903393714 3339136.0154795544),POLYGON ((2300917.4903393714 3339136.015479554...,0.0


### Numbers of building that are affected by war fire (using Intersection of war_fires and buildings)

In [27]:
# Create a sum of the total number of buildings affected by the war fires
geo_warfires['building_count'].sum()

3724.0

In [28]:
# Get the earliest and latest dates of the war fires
geo_warfires.date.min(), geo_warfires.date.max()

('2022-02-24', '2024-01-21')

### Showing building_count based on geometry

In [29]:

# Filter out data to consider Lyman's district
filtered_data = geo_warfires[geo_warfires["ADM3_EN"] == "Donetska"]

n = folium.Map(location=[filtered_data['LATITUDE'].mean(), filtered_data['LONGITUDE'].mean()], zoom_start=10)

for index, row in filtered_data.iterrows():
    folium.Marker(
        [row['LATITUDE'], row['LONGITUDE']],
        popup=f"Building Count: {row['building_count']}"
    ).add_to(n)

n.save("./notebooks/figures/ukr_warfire_build_count_markers2.html")


In [25]:
filtered_data.shape

(241, 24)

### War fires with buff and number of buildings are affected

In [5]:
import pandas as pd
import folium


# Filter out rows where building_count is greater than 0 and based on geometry_buff
filtered_data = geo_warfires[(geo_warfires['building_count'] > 0) & (geo_warfires['geometry_buff'].notnull())]

m = folium.Map(location=[filtered_data['LATITUDE'].mean(), filtered_data['LONGITUDE'].mean()], zoom_start=10)

for index, row in filtered_data.iterrows():
    folium.Circle(
        radius=10,  # Adjust the radius of the circle as needed
        location=[row['LATITUDE'], row['LONGITUDE']],
        popup=f"Building Count: {row['building_count']}",
        color='crimson',  # Color of the circle outline
        fill=True,
        fill_color='crimson'  # Fill color of the circle
    ).add_to(m)

m.save("./notebooks/figures/ukr_warfire_build_count_markers_with_geobuff.html")

m.save("./notebooks/figures/ukr_warfire_build_count.html")


In [6]:
# Convert to a python script and save in local folder
!jupyter nbconvert --to script ./notebooks/*.ipynb --output-dir='./notebooks//python_scripts'

[NbConvertApp] Converting notebook ./notebooks/24_01_22-hh-building_count_full.ipynb to script
/Users/harryhopalot/Documents/GitHub/dssd_Insulate_ukraine/.venv/lib/python3.11/site-packages/nbformat/__init__.py:93: MissingIDFieldWarning: Code cell is missing an id field, this will become a hard error in future nbformat versions. You may want to use `normalize()` on your notebooks before validations (available since nbformat 5.1.4). Previous versions of nbformat are fixing this issue transparently, and will stop doing so in the future.
  validate(nb)
[NbConvertApp] Writing 3660 bytes to notebooks/python_scripts/24_01_22-hh-building_count_full.py
[NbConvertApp] Converting notebook ./notebooks/24_01_22-hh-buildings_impacted_by_war_fires.ipynb to script
/Users/harryhopalot/Documents/GitHub/dssd_Insulate_ukraine/.venv/lib/python3.11/site-packages/nbformat/__init__.py:93: MissingIDFieldWarning: Code cell is missing an id field, this will become a hard error in future nbformat versions. You ma